# Day 7

We need to execute a number of steps in the right order.

In [ ]:
import regex as re
import numpy as np
import pandas as pd

In [ ]:
# Sample data as provided in question
data= """
Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.
""".strip().splitlines()

In [ ]:
# We're now going to start storing the data in a file to avoid having to paste into the main document
# Make sure you create a file with the name below and save the real problem output there. 
# If you want to run just the sample data, skip this block
with open("./07-kws.txt", "r") as FILE:
    data = FILE.read().strip().splitlines()

In [ ]:
data[:5]

In [ ]:
pattern = re.compile("([A-Z]) must .* step ([A-Z])")
steps = []
for line in data:
    match = pattern.search(line)
    if match:
        steps.append(dict(step=match[1], dependant=match[2]))
        
steps = pd.DataFrame(steps, columns=["step","dependant"])
steps.head(5)

In [ ]:
# Now we find if there are any steps that aren't a dependant
order = []

df = steps
while df.shape[0] > 0:
    execute_next = df[~df["step"].isin(df["dependant"])]["step"].drop_duplicates().sort_values().tolist()
    if len(execute_next)>0: execute_next = execute_next[:1]
    order += execute_next
    df = df[~df["step"].isin(execute_next)]
df
del df

# Final steps
order += steps[~steps["dependant"].isin(steps["step"])]["dependant"].drop_duplicates().sort_values().tolist()

"".join(order)

# Part 2 

In [ ]:
steps = {}
for line in data:
    match = pattern.search(line)
    if match:
        step_id = match[1]
        dep_id = match[2]
        
    step = steps.get(step_id, {"id": step_id})
    steps[step_id] = step
    
    dep  = steps.get(dep_id, {"id": dep_id})
    steps[dep_id] = dep

    depends_on = dep.get("depends_on", [])
    dep["depends_on"] = depends_on
    depends_on.append(step)
    
def is_available(step):
    """ 
        A step is available if it:
            * not started
            * has no entries it depends_on
            * all the depends_on are complete """

    started = step.get("started", None)
    if started is not None: return False

    depends_on = step.get("depends_on", None)
    if depends_on is None: return True

    return all([d.get("complete", False) for d in depends_on])
    
def find_available():
    return list(filter(is_available, steps.values()))

worker_count = 5
time_penalty = 60

workers = []
completed = []
for time in range(0, 1000):
    # Check any that are waiting
    available = find_available()
    available = sorted(available, key=lambda v: v["id"])

    while len(workers) < worker_count and len(available)>0:
        step = available[0]
        available.remove(step)
        step["started"] = time
        workers.append(step)

    print(time, {w["id"]: w["started"] for w in workers}, "".join([s["id"] for s in completed]),
         "Waiting:", "".join([s["id"] for s in available]))

    # Complete any that are completing this time
    for step in workers:
        if time >= step["started"] + ord(step["id"]) - 65 + time_penalty:
            step["complete"] = time
            workers.remove(step)
            completed.append(step)
            
    if len(completed) == len(steps):
        print("Finished in {} seconds".format(time+1))
        break
        
